In [1]:
import pandas as pd
import numpy as np

In [100]:
import os
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize
import string

In [133]:
data=pd.read_csv('Question Similarity Percentage/train.csv',encoding='latin-1')
# data=pd.read_csv('train.csv',encoding='latin-1')

In [134]:
data.head()

,id,qid1,qid2,question1,question2
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?


In [135]:
data.shape

(404290, 5)

In [136]:
data.isnull().sum()

id           0
qid1         0
qid2         0
question1    1
question2    2
dtype: int64

In [137]:
data.fillna('', inplace=True)

In [138]:
data.isnull().sum()

id           0
qid1         0
qid2         0
question1    0
question2    0
dtype: int64

The original problem is Quora Question pair similarity which is a supervised learning problem. Now, we don't have the label present in our data. So, we can not leverage the power of supervised or unsupervised machine learning algorithms to predict the percentage similarity between the two problems.

Hence we'll create a simple function to compute the similarity based on the common words present

In [139]:
# def normalized_word_share(row):
#     w1 = set(map(lambda word: word.lower().strip(), row['question1'].split(" ")))
#     w2 = set(map(lambda word: word.lower().strip(), row['question2'].split(" ")))    
#     return 1.0 * len(w1 & w2)/(len(w1) + len(w2))
# data['word_share'] = data.apply(normalized_word_share, axis=1)

In [110]:
data.dtypes

id            int64
qid1          int64
qid2          int64
question1    object
question2    object
dtype: object

In [5]:
# First, we'll create a function to clean the data

# Then, we'll build another function to compute similarity using a simple mathematical formula

In [140]:
def token(row):

    st1=re.sub(r'[?|!|\'|"|#|@|$|%|:|;|.]',r'', row)
    st2=re.sub(r'[-|,|)|(|\|/|&|_]',r' ',st1)
    st3=re.sub(r'[1|2|3|4|5|6|7|8|9|0]', r'', st2)
    st3 = re.sub('[^a-zA-Z]', ' ', st3)
    st3=st3.lower()
    
    # Using word_tokenize to tokenize sentence into words

    tokens = word_tokenize(st3, "english")
    
    return tokens

In [141]:
def token_stop(row):

    st1=re.sub(r'[?|!|\'|"|#|@|$|%|:|;|.]',r'', row)
    st2=re.sub(r'[-|,|)|(|\|/|&|_]',r' ',st1)
    st3=re.sub(r'[1|2|3|4|5|6|7|8|9|0]', r'', st2)
    st3 = re.sub('[^a-zA-Z]', ' ', st3)
    st3=st3.lower()

    # Using word_tokenize to tokenize sentence into words

    tokens = word_tokenize(st3, "english")
    
    # Removing Stop Words
    final_words = []

    for word in tokens:
        if word not in stopwords.words('english'):
            final_words.append(word)
    
    return final_words

In [142]:
data['token_q1']=data['question1'].apply(token)
data['token_q2']=data['question2'].apply(token)

data['token_q1_stop']=data['question1'].apply(token_stop)
data['token_q2_Stop']=data['question2'].apply(token_stop)

In [91]:
# def num_common(row1, row2):
#     lst = [value for value in row1 if value in row2] 
#     return len(lst)

In [92]:
# lst1=['step', 'step', 'guide', 'invest', 'share', 'market', 'india']
# lst2=['step', 'step', 'guide', 'invest', 'share', 'market']

# num_common(lst1, lst2)

In [93]:
# def total(row1, row2):
#     return len(row1)+len(row2)

In [143]:
data.head()

,id,qid1,qid2,question1,question2,token_q1,token_q2,token_q1_stop,token_q2_Stop
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...","[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]"
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,"[what, is, the, story, of, kohinoor, koh, i, n...","[what, would, happen, if, the, indian, governm...","[story, kohinoor, koh, noor, diamond]","[would, happen, indian, government, stole, koh..."
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,"[how, can, i, increase, the, speed, of, my, in...","[how, can, internet, speed, be, increased, by,...","[increase, speed, internet, connection, using,...","[internet, speed, increased, hacking, dns]"
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,"[why, am, i, mentally, very, lonely, how, can,...","[find, the, remainder, when, math, math, is, d...","[mentally, lonely, solve]","[find, remainder, math, math, divided]"
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,"[which, one, dissolve, in, water, quikly, suga...","[which, fish, would, survive, in, salt, water]","[one, dissolve, water, quikly, sugar, salt, me...","[fish, would, survive, salt, water]"


In [148]:
def similarity(row):
    lst1=set(row['token_q1'])
    lst2=set(row['token_q2'])
    com=2*len(lst1 & lst2)
    total=len(lst1)+len(lst2)
    if total!=0:
        sim=com/total
    else:
        sim=0
    return sim*100

def similarity_stop(row):
    lst1=set(row['token_q1_stop'])
    lst2=set(row['token_q2_Stop'])
    com=2*len(lst1 & lst2)
    total=len(lst1)+len(lst2)
    if total!=0:
        sim=com/total
    else:
        sim=0
    return sim*100

In [149]:
data['sim']=data.apply(similarity, axis=1)
data['sim_stop']=data.apply(similarity_stop, axis=1)

In [150]:
data.head()

,id,qid1,qid2,question1,question2,token_q1,token_q2,token_q1_stop,token_q2_Stop,sim,sim_stop
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,"[what, is, the, step, by, step, guide, to, inv...","[what, is, the, step, by, step, guide, to, inv...","[step, step, guide, invest, share, market, india]","[step, step, guide, invest, share, market]",95.652174,90.909091
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,"[what, is, the, story, of, kohinoor, koh, i, n...","[what, would, happen, if, the, indian, governm...","[story, kohinoor, koh, noor, diamond]","[would, happen, indian, government, stole, koh...",58.333333,53.333333
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,"[how, can, i, increase, the, speed, of, my, in...","[how, can, internet, speed, be, increased, by,...","[increase, speed, internet, connection, using,...","[internet, speed, increased, hacking, dns]",33.333333,36.363636
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,"[why, am, i, mentally, very, lonely, how, can,...","[find, the, remainder, when, math, math, is, d...","[mentally, lonely, solve]","[find, remainder, math, math, divided]",0.000000,0.000000
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,"[which, one, dissolve, in, water, quikly, suga...","[which, fish, would, survive, in, salt, water]","[one, dissolve, water, quikly, sugar, salt, me...","[fish, would, survive, salt, water]",40.000000,26.666667


In [151]:
# Hence, we have created a function to compute similarity

In [152]:
# Lets create a function/pipeline so that given two questions as string, we can compute the similarity

In [153]:
q1='How do I read and find my YouTube comments?'
q2='How can I see all my Youtube comments?'

In [154]:
# To find similarity with stopwords
def sim(q1,q2):
    lst1=set(token(q1))
    lst2=set(token(q2))
    com=2*len(lst1 & lst2)
    total=len(lst1)+len(lst2)
    if total!=0:
        sim=com/total
    else:
        sim=0
    return sim*100

sim(q1,q2)

58.82352941176471

In [155]:
# To find similarity without stopwords
def sim_stop(q1,q2):
    lst1=set(token_stop(q1))
    lst2=set(token_stop(q2))
    com=2*len(lst1 & lst2)
    total=len(lst1)+len(lst2)
    if total!=0:
        sim=com/total
    else:
        sim=0
    return sim*100

sim_stop(q1,q2)

57.14285714285714